In [1]:
% load_ext autoreload
% autoreload 2

In [16]:
from models import pretrained
from data.civilcomments import CivilComments
import torch
from torchmetrics import Accuracy
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
bert = pretrained.distillbert_on_civilcomments(device='cuda:0')
# bert = bert.to(device='cuda:0')
print(bert.device)
bert.eval()

cuda:0


DistilBertClassifier(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

In [23]:
dm = CivilComments(batch_size=32)
acc = Accuracy()

In [24]:
for batch in tqdm(dm.train_dataloader()):
    with torch.no_grad():
        y = bert(batch[0].to(device='cuda:0')).cpu()
        acc(y, batch[1])

/tmp/ipykernel_29706/569867731.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm(dm.train_dataloader()):


  0%|          | 0/8095 [00:00<?, ?it/s]

In [25]:
acc.compute()

tensor(0.8582)

In [26]:
acc.reset()
for batch in tqdm(dm.val_dataloader()):
    with torch.no_grad():
        y = bert(batch[0].to(device='cuda:0')).cpu()
        acc(y, batch[1])
print(acc.compute())

/tmp/ipykernel_29706/1858523385.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm(dm.val_dataloader()):


  0%|          | 0/1412 [00:00<?, ?it/s]

tensor(0.8517)


In [27]:
acc.reset()
for batch in tqdm(dm.test_dataloader()):
    with torch.no_grad():
        y = bert(batch[0].to(device='cuda:0')).cpu()
        acc(y, batch[1])
print(acc.compute())

/tmp/ipykernel_29706/869455391.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm(dm.test_dataloader()):


  0%|          | 0/4181 [00:00<?, ?it/s]

tensor(0.8486)


In [44]:
dm = CivilComments(batch_size=128, return_meta=True)
dm.test_dl.dataset.transform = lambda x: x

In [4]:
tr_meta = [batch[-1] for batch in tqdm(dm.train)]
tr_meta = torch.stack(tr_meta)

  0%|          | 0/259038 [00:00<?, ?it/s]

In [20]:
def bar(dataloader):
    meta = [batch[-1] for batch in tqdm(dataloader)]
    meta = torch.cat(meta)
    plt.bar(range(meta.shape[1]), [(meta[:, i] == 1).float().mean().item() for i in range(meta.shape[1])])

In [46]:
from wilds.common.grouper import CombinatorialGrouper

In [47]:
g = CombinatorialGrouper(dm.dataset, ['christian'])

tensor([2])